In [1]:
# Task:
# An SVD Recommender that predicts the rating a user will give to a movie
# based on the user's own ratings and other users' rating data.

# Use only 'rating' as the data, avoid 'tags' and 'genre'

# 80/20, train/test split. Additionally, do a temporal split. 


In [2]:
# imports
import pandas as pd
from numpy.linalg import svd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
import math
import random

In [3]:
# read data
movies = 'data/movielens-latest-small/movies.csv'
ratings = 'data/movielens-latest-small/ratings.csv'

# to dataframes
df_movies = pd.read_csv(movies)
df_ratings = pd.read_csv(ratings)

# inspect them
display('Movies')
display(df_movies.head())
display('Ratings')
display(df_ratings.head())

'Movies'

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


'Ratings'

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
# 80/20, train/test split
df_ratings_x = df_ratings[['userId', 'movieId', 'timestamp']]
df_ratings_y = df_ratings[['rating', 'timestamp']]

x_train, x_test, y_train, y_test = train_test_split(df_ratings_x, df_ratings_y, test_size=0.2, random_state=1)
print(f"Training rows = {x_train.shape[0]}")
print(f"Testing rows = {x_test.shape[0]}")

#display(x_train.head())
#display(x_test.head())
#display(y_train.head())
#display(y_test.head())

# temporal split
tscv = TimeSeriesSplit(n_splits=2, test_size=20000)
for i, (train_index, test_index) in enumerate(tscv.split(x_train, y_train)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

print(f"Training rows temporal split = {train_index.shape[0]}")
print(f"Testing rows temporal split = {test_index.shape[0]}")

Training rows = 80668
Testing rows = 20168
Fold 0:
  Train: index=[    0     1     2 ... 40665 40666 40667]
  Test:  index=[40668 40669 40670 ... 60665 60666 60667]
Fold 1:
  Train: index=[    0     1     2 ... 60665 60666 60667]
  Test:  index=[60668 60669 60670 ... 80665 80666 80667]
Training rows temporal split = 60668
Testing rows temporal split = 20000


In [5]:
# Consider reviews from users with more than 50 reviews
#usercount = df_ratings[['movieId','userId']].groupby('userId').count()
#display(usercount.head())

In [6]:
# Source for SVD stuff: https://machinelearningmastery.com/using-singular-value-decomposition-to-build-a-recommender-system/
# Build a pivot table with movieIds as columns 
# and users and their ratings as rows
rating_matrix = df_ratings.pivot(index="userId", columns="movieId", values="rating").fillna(0)
display(rating_matrix.head())
matrix = rating_matrix.values
display(matrix)


movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


array([[4. , 0. , 4. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       [0. , 0. , 0. , ..., 0. , 0. , 0. ],
       ...,
       [2.5, 2. , 2. , ..., 0. , 0. , 0. ],
       [3. , 0. , 0. , ..., 0. , 0. , 0. ],
       [5. , 0. , 0. , ..., 0. , 0. , 0. ]])

In [7]:
# Singular value decomposition
u, s, vh = np.linalg.svd(matrix, full_matrices=False)
# We know that the columns of vh are movies
# The rows of u are users

In [8]:
def cosine_similarity(v,u):
    return (v @ u)/ (np.linalg.norm(v) * np.linalg.norm(u))

In [9]:
# Generate similarities to all others users for a single user
def get_similarities(user_index, u):
    similarities = []
    
    for row in range(0, u.shape[0]):
        #if row != user_index:
        similarity  = cosine_similarity(u[user_index, :], u[row, :])
        similarities.append(similarity)
    
    return similarities

similarities = get_similarities(0, u)
#display(similarities)

# Predict a rating using the U matrix from the SVD operation
def predict_rating(user_index, movie_index, ratings_matrix, u, k=5):
    # Get the ratings for the target user and the movie
    target_user_ratings = ratings_matrix[user_index, :]
    movie_ratings = ratings_matrix[:, movie_index]

    # Calculate cosine similarity between the target user and all other users
    # 1-d matrix of length users
    similarities  = get_similarities(user_index, u)
    
    # Sort the similarities and get the indices of the nearest neighbors
    nearest_neighbors = np.argsort(similarities)[::-1][1::]  # Exclude the target user
    
    # Calculate weighted average of ratings from nearest neighbors
    weighted_ratings = 0
    total_similarity = 0
    for neighbor_index in nearest_neighbors:
        neighbor_similarity = similarities[neighbor_index]
        neighbor_rating = ratings_matrix[neighbor_index, movie_index]
        if neighbor_rating != 0:  # Ignore if neighbor hasn't rated the movie
            weighted_ratings += neighbor_similarity * neighbor_rating
            total_similarity += neighbor_similarity
    
    # Predict the rating for the target user
    if total_similarity != 0:
        predicted_rating = weighted_ratings / total_similarity
    else:
        predicted_rating = 0  # In case none of the nearest neighbors have rated the movie
    
    return predicted_rating

predicted_rating = predict_rating(user_index = 0, movie_index = 0, ratings_matrix = matrix, u = u)
print("Predicted rating:", predicted_rating)

Predicted rating: 1.062015503875969


In [18]:
# K-means RMSE Clustering 
# select k random data points (user indexes in u) as initial cluster centers C_1, ..., C_k
k = 5
random.seed(1)

def initialize_centroids(data_points, k):
    # Select k random indexes from the data points list
    centroid_indexes = random.sample(range(data_points.shape[0]), k)
    
    # Construct the list of centroids using the selected indexes
    centroids = [data_points[i] for i in centroid_indexes]
    
    return centroids

def assign_point_to_centroid(point, closest_centroid, assignments):
    # convert centroid to tuple for hashability
    closest_centroid_tuple = tuple(closest_centroid)
    
    display(closest_centroid)
    display(assignments)
    if closest_centroid_tuple not in assignments:
        assignments[closest_centroid_tuple ] = [point]
    else:
        assignments[closest_centroid_tuple ].append(point)

def update_centroid_position(centroid, new_position):
    centroid[:] = new_position
    
initial_centroids = initialize_centroids(u, k)
assignments = {}

# for each p (user) in u, map p_i to its nearest cluster center C_j 
for i, p in enumerate(u):
    # Find the closest centroid to the current data point
    closest_centroid = max(initial_centroids, key=lambda centroid: cosine_similarity(p, centroid))
    
    # Assign the point to the closest centroid    
    assign_point_to_centroid(p, closest_centroid, assignments)
    break

for centroid in initial_centroids:
    # Update centroid position by taking the mean of assigned data points
    
    # convert centroid to tuple for hashability
    centroid_tuple = tuple(centroid)
    
    assigned_points = assignments[centroid_tuple]  # get all data points assigned to a centroid
    if assigned_points:
        new_position = mean(assigned_points)
        
         # calculate Euclidean distance
        difference = np.linalg.norm(centroid - new_position)
        display(difference)
        
        update_centroid_position(centroid, new_position)
        break
        
       

array([-8.58065297e-03,  5.91195617e-03,  1.22308431e-02, -1.13904883e-02,
       -2.34989304e-02, -7.28491110e-03,  1.23433603e-02, -2.13440229e-02,
        2.80708852e-02,  1.55837541e-03,  2.22658774e-03,  9.38638279e-03,
       -8.94429705e-03,  1.27634859e-02,  1.57755362e-02,  9.06190097e-03,
        1.22891899e-02,  9.98297431e-03, -5.96683857e-03,  1.28640381e-02,
        1.53656035e-02,  1.19677609e-02,  3.98036514e-03,  7.81239794e-03,
       -4.79812652e-03, -4.89367945e-03, -8.11135089e-03,  1.25321189e-02,
        7.52847666e-03,  1.87589569e-02,  1.75420479e-02, -1.22629739e-02,
       -3.49279690e-03,  1.42268216e-02, -1.40639639e-02, -1.86853195e-02,
        6.93307812e-03, -1.62249453e-03,  4.84139367e-04, -1.81760651e-02,
        1.23724982e-02, -8.01954942e-03,  3.32582804e-03,  9.08077896e-03,
       -1.05897594e-02, -9.54093819e-04, -3.07715724e-03, -3.61065794e-03,
       -1.77501433e-02,  4.57186085e-03, -4.81890639e-03, -6.27942728e-03,
       -1.58136741e-02,  

{}

KeyError: (-0.0018288086744643725, -0.0021490221680925915, -0.003172909059712625, -0.0017594331713546606, 0.00011317166677794145, 0.000443520168753731, 0.0022473445936203786, -0.0010652385590969836, -0.002645763924895561, 0.00544033281487763, -4.679681680276613e-05, 0.0005802457720369058, -0.005152682754894517, 0.0003809002527677763, -0.0015549816290862358, 0.0017334989354834292, -0.0036127634769211967, 0.002348353228834049, -0.0023735408468744134, -0.006205691601139467, -0.002324729617467638, 0.009088688588650544, -0.0009527461229227132, 0.003226851193006082, -0.0010767574957852755, -0.0057991495583760745, -0.0012456310233366936, -0.0014081808778273436, -0.00470886675100656, 0.0005877087249213844, 0.003479487149897314, 0.008131858765849535, -0.0029134975431994465, 0.007016994151983856, -0.00025525612906256764, 0.001177190932352785, 0.0035700311713014994, -0.00132428163316986, -0.003718701439533457, 0.001725648423079484, 0.0031272729978018728, -0.005559343511407475, 0.00261473698768615

In [11]:
# Evaluate rmse

# for each user, predict rating for every movie
# check distance from actual, if review exists
total_error_squared = 0

for row in range(0, u.shape[0]):
    user_total_error_squared = 0
    user_evaluations = 0
    
    for col in range(0, u.shape[1]):
        predicted_rating = predict_rating(user_index = row, movie_index = col, ratings_matrix = matrix, u = u)
        actual_rating = matrix[row][col]
        if actual_rating != 0:
            user_evaluations += 1
            user_total_error_squared += (predicted_rating - actual_rating)**2
    
    user_total_error = math.sqrt(user_total_error_squared)
    print("User total error:", user_total_error)
    print("User evaluations:", user_evaluations)
    #break
total_error = math.sqrt(total_error_squared)
print("Total error:", total_error)

User total error: 16.05560880831214
User evaluations: 42
User total error: 4.013240739342899
User evaluations: 2
User total error: 6.988303485138862
User evaluations: 5


KeyboardInterrupt: 

In [11]:
# Find the highest similarity

lowest_similarity = np.inf
highest_similarity = -np.inf
highest_sim_col = -1
for col in range(1,vh.shape[1]):
    similarity = cosine_similarity(vh[:,0], vh[:,col])
    if similarity > highest_similarity:
        highest_similarity = similarity
        highest_sim_col = col
    if similarity < lowest_similarity:
        lowest_similarity = similarity

print("highest_similarity is %s" % (highest_similarity))
print("lowest_similarity is %s" % (lowest_similarity))

print("Column %d (movie id %s) is most similar to column 0 (movie id %s)" %
        (highest_sim_col, rating_matrix.columns[col], rating_matrix.columns[0])
)

highest_similarity is 0.08144239149393877
lowest_similarity is -0.06146904339534752
Column 542 (movie id 193609) is most similar to column 0 (movie id 1)


In [12]:
# Predict a rating
def predict_rating(user_id, movie_id, ratings_matrix, k=5):
    # Get the ratings for the target user and the movie
    target_user_ratings = ratings_matrix[user_id, :]
    movie_ratings = ratings_matrix[:, movie_id]
    
    predicted_rating = 0
    total_similarity = 0
    
     # Calculate cosine similarity between the target user and all other users
    for i, user_ratings in enumerate(ratings_matrix):
        if i == user_id:
            continue  # Skip the target user
        
        similarity = cosine_similarity(target_user_ratings, user_ratings)
        neighbor_rating = ratings_matrix[i, movie_id]
        
        if neighbor_rating != 0:  # Ignore if neighbor hasn't rated the movie
            predicted_rating += similarity * neighbor_rating
            total_similarity += similarity

    if total_similarity != 0:
        predicted_rating /= total_similarity
    
    return predicted_rating

# Example usage
# Assuming ratings_matrix is your pivot matrix where rows represent users and columns represent movies
# and user_id and movie_id are valid indices in your matrix
predicted_rating = predict_rating(user_id = 0, movie_id = 0, ratings_matrix = matrix)
print("Predicted rating:", predicted_rating)

Predicted rating: 3.9086166688351893
